<a href="https://colab.research.google.com/github/slipaway/melonplaylist/blob/%EC%84%B1%ED%99%98/Word2vec_%EB%AA%B0%EB%B9%B5%EB%B2%84%EC%A0%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

fatal: destination path 'khaiii' already exists and is not an empty directory.
ERROR: Operation cancelled by user
mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test fma_compiles
-- Performing Test fma_compiles - Success
-- Performing Test fma_runs
-- Performing Test fma_runs - Success
-- [khaiii] fused multiply add option enabled
-- [hunter] Calculating Toolchain-SHA1
Traceback (most

KeyboardInterrupt: ignored

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/zmq/eventloop/zmqstream.py", line 438, in _run_callback
    callback(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 636, in _abort_queues
    self._abort_queue(stream)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 654, in _abort_queue
    sel

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import timedelta, datetime
import glob
import os
import re
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
from collections import Counter
from typing import *
from khaiii import KhaiiiApi

In [ ]:
folder = '/content/gdrive/My Drive/melon'
filelist = os.chdir(folder)

In [ ]:
genre_gn_all=pd.read_json('genre_gn_all.json', typ='series')
train=pd.read_json('train.json')
song_meta=pd.read_json('song_meta.json')

In [ ]:
!git clone https://github.com/slipaway/melonplaylist.git

In [ ]:
genre_gn_all.head()

In [ ]:
genre_gn_all = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})

In [ ]:
gnr_code=genre_gn_all[genre_gn_all['gnr_code'].str[-2:]=='00']

In [ ]:
#대장르 장르코드 리스트
gnr_code

In [ ]:
#곡 아이디(id)와 대분류 장르코드 리스트(song_gn_gnr_basket) 추출

song_gnr_map = song_meta.loc[:, ['id', 'song_gn_gnr_basket']]

#unnest song_gn_gnr_basket
song_gnr_map_unnest = np.dstack(
    (
        np.repeat(song_gnr_map.id.values, list(map(len, song_gnr_map.song_gn_gnr_basket))),
        np.concatenate(song_gnr_map.song_gn_gnr_basket.values)
    )
)

#unnested 데이터프레임 생성 : song_gnr_map

song_gnr_map = pd.DataFrame(data = song_gnr_map_unnest[0], columns=song_gnr_map.columns)
song_gnr_map['id']=song_gnr_map['id'].astype(str)
song_gnr_map.rename(columns={'id' : 'song_id', 'song_gn_gnr_basket' : 'gnr_code'}, inplace=True)

#unnest 객체 제거
del song_gnr_map_unnest

In [ ]:
plylst_date = train[['updt_date', 'songs']]
plylst_date_unnest = np.dstack(
    (
        np.repeat(plylst_date.updt_date.values, list(map(len, plylst_date.songs))), 
        np.concatenate(plylst_date.songs.values)
    )
)
plylst_date = pd.DataFrame(data=plylst_date_unnest[0], columns = plylst_date.columns)

del plylst_date_unnest

In [ ]:
plylst_date["updt_date"] = plylst_date["updt_date"].map(lambda x: x[:4] + x[5:7] + x[8:10])
song_date = song_meta[["id","issue_date"]]
plylst_song_date = pd.merge(plylst_date, song_date, left_on="songs", right_on="id", how='left')
plylst_song_date["issue_date"] = plylst_song_date["issue_date"].astype(str)

plylst_song_date["strange"] = plylst_song_date["updt_date"] < plylst_song_date["issue_date"]
strange_songs = plylst_song_date[plylst_song_date["strange"] == True].drop('songs', axis=1)

In [ ]:
strange_songs

In [ ]:
strange_songs=strange_songs.drop_duplicates(['id'])
len(strange_songs)

In [ ]:
strange_songs_list=strange_songs['id'].values.astype(str)
strange_song_gnr_map=song_gnr_map[song_gnr_map['song_id'].isin(strange_songs_list)]

In [ ]:
strange_cnt=strange_song_gnr_map.groupby('gnr_code').count()
strange_cnt=pd.merge(strange_cnt, gnr_code, on='gnr_code')

In [ ]:
strange_cnt

In [ ]:
strange_songs=strange_songs.drop_duplicates(['id'])
print(strange_songs)

In [ ]:
strange_id=strange_songs.loc[:, 'id']
strange_id
strange_id_list=list(strange_id)

In [ ]:
train_id=train.loc[:, 'id']

In [ ]:
song_meta_id=song_meta.loc[:, 'id']
print(song_meta_id.head())

In [ ]:
strange_songs

In [ ]:
#플레이리스트 아이디(id)와 수록곡(songs) 추출
plylst_song_map=train[['id', 'songs']]

#unnest songs

plylst_song_map_unnest=np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))),
        np.concatenate(plylst_song_map.songs.values)
    )
)

#unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map=pd.DataFrame(data=plylst_song_map_unnest[0], columns=plylst_song_map.columns)
plylst_song_map['id']=plylst_song_map['id'].astype(str)
plylst_song_map['songs']=plylst_song_map['songs'].astype(str)

del plylst_song_map_unnest

In [ ]:
#플레이리스트에 수록된 곡의 총 개수(중복 포함)

len(plylst_song_map)

In [ ]:
#플레이리스트에 strange 곡 제거한 후, 수록된 곡의 총 개수(중복 포함)

plylst_song_map_remove=plylst_song_map[np.logical_not(plylst_song_map.songs.isin(strange_songs_list))]
plylst_song_map_remove.shape

In [ ]:
#train set에 새로운 column : remove_songs 생성
#remove_songs : strange 곡 제거한 후 다시 만든 플레이리스트

plylst_song_group=plylst_song_map_remove.groupby('id')['songs'].apply(list).reset_index(name='remove_songs')
train['id']=train['id'].astype(str)
train_remove = pd.merge(train, plylst_song_group)

train_remove.head()

In [ ]:
def re_sub(series: pd.Series) -> pd.Series:
  series = series.str.replace(pat=r'[ㄱ-ㅎ]', repl=r'', regex=True) #ㅋ 제거용
  series = series.str.replace(pat=r'[^\w\s]', repl=r'', regex=True) #특수문자 제거용
  series = series.str.replace(pat=r'[ ]{2,}', repl=r' ', regex=True) #공백 제거
  series = series.str.replace(pat=r'[\u3000]+', repl=r'', regex=True) #u3000제거
  return series

def flatten(list_of_list : List) -> List:
  flatten = [j for i in list_of_list for j in i]
  return flatten

def get_token(title: str, tokenizer)-> List[Tuple]:

  if len(title)==0 or title== ' ' : #제목이 공백인 경우 tokenizer 에러 발생
    return []
  
  result = tokenizer.analyze(title)
  result = [(morph.lex, morph.tag) for split in result for morph in split.morphs] # (형태소, 품사) 튜플의 리스트
  return result

def get_all_tags(df) -> List:
  tag_list = df['tags'].values.tolist()
  tag_list=flatten(tag_list)
  return tag_list


In [ ]:
tokenizer = KhaiiiApi()
all_tag = get_all_tags(train)
token_tag = [get_token(x, tokenizer) for x in all_tag] #태그를 형태소 분석

In [ ]:
token_tag[:10]

In [ ]:
token_itself=list(filter(lambda x: len(x)==1, token_tag)) # 태그 자체가 형태소여서 분리되지 않는 태그만 고름
token_itself=flatten(token_itself)
flatten_token=flatten(token_tag)

In [ ]:
print('%-23s'%'# of original tag is', f'{len(all_tag):8,}')
print('%-23s'%'# of morpheme itself is', f'{len(token_itself):8,}')
print('%-23s'%'# of total token is', f'{len(flatten_token):8,}')

In [ ]:
#counting part of speech (pos)
pos=[x[1] for x in token_itself]
pos_count=Counter(pos)
popular_pos = pos_count.most_common()

In [ ]:
#tag 분류표
objects=[x[0] for x in popular_pos]
y_pos=np.arange(len(objects))
performance=[x[1] for x in popular_pos]

plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Usage')
plt.title('Part of Speech - Tags')
plt.show()

In [ ]:
#플레이리스트 제목 형태소 분석

train['plylst_title']=re_sub(train['plylst_title'])
train.loc[:, 'ply_token']=train['plylst_title'].map(lambda x: get_token(x, tokenizer))

In [ ]:
using_pos=['NNG', 'SL', 'NNP', 'MAG', 'SN'] #일반 명사, 외국어, 고유 명사, 일반 부사, 숫자
train['ply_token']=train['ply_token'].map(lambda x: list(filter(lambda x: x[1] in using_pos, x)))


In [ ]:
unique_tag=set(token_itself)
unique_word=[x[0] for x in unique_tag]

#train 데이터의 plylst title 형태소 분리

In [ ]:
#정답 tag에 나온 형태소만 남겨두기
train['ply_token']=train['ply_token'].map(lambda x: list(filter(lambda x: x[0] in unique_word, x)))
train.head(10)

In [ ]:
train_eda = pd.merge(train, plylst_song_group)
train_eda

In [ ]:
wv_sent = train_eda[['ply_token', 'tags', 'remove_songs']]
wv_sent

In [ ]:
#Word2vec 이용을 위해 ply_token data를 전처리
z = []
y = []
for i in range(len(wv_sent['ply_token'])):
  z = []
  for j in range(len(wv_sent['ply_token'][i])):
    x = wv_sent['ply_token'][i][j]
    x = list(x)
    del x[1]
    z.insert(j,"".join(x))
  y.insert(i,z)
y

In [ ]:
wv_sent['ply_token_new'] = y
wv_sent = wv_sent[['ply_token_new', 'tags', 'remove_songs']]
wv_sent # Word2vec 이용을 위해 필요한 data들만 모아둔 dataframe

In [ ]:
train_token_list = np.dstack(
    (
        np.concatenate(wv_sent.ply_token_new.values)
    )
)
train_token_list = pd.DataFrame(train_token_list[0][0], columns = ['token'])
train_token_list = list(train_token_list['token']) # train data에 존재하는 모든 플레이리스트 제목 형태소들의 list

train_tag_list = np.dstack(
    (
        np.concatenate(wv_sent.tags.values)
    )
)
train_tag_list = pd.DataFrame(train_tag_list[0][0], columns = ['tag'])
train_tag_list = list(train_tag_list['tag']) # train data에 존재하는 모든 태그들의 list

train_song_list = np.dstack(
    (
        np.concatenate(wv_sent.remove_songs.values)
    )
)
train_song_list = pd.DataFrame(train_song_list[0][0], columns = ['song'])
train_song_list = list(train_song_list['song']) # train data에 존재하는 노래들의 list

In [ ]:
train_token_list


In [ ]:
train_tag_list


In [ ]:
train_songs_list


In [ ]:
#Word2vec 사용을 위해 플레이리스트 별로 제목의 형태소와 태그, 단어를 한 문장으로 갖도록 전처리
w = []
for i in range(len(wv_sent)):
  w.insert(i,wv_sent['ply_token_new'][i] + wv_sent['tags'][i] + wv_sent['remove_songs'][i])
train_eda_fin = w
train_eda_fin

In [ ]:
from gensim.test.utils import get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec
class EpochSaver(CallbackAny2Vec):
     '''Callback to save model after each epoch.'''

     def __init__(self, path_prefix):
         self.path_prefix = path_prefix
         self.epoch = 0

     def on_epoch_end(self, model):
         output_path = get_tmpfile('{}_epoch{}.model'.format(self.path_prefix, self.epoch))
         model.save(output_path)
         self.epoch += 1

class EpochLogger(CallbackAny2Vec):
     def __init__(self):
         self.epoch = 0

     def on_epoch_begin(self, model):
         print("Epoch #{} start".format(self.epoch))

     def on_epoch_end(self, model):
         print("Epoch #{} end".format(self.epoch))
         self.epoch += 1


In [ ]:
from gensim.models import Word2Vec
epoch_logger = EpochLogger()
wv_model = Word2Vec(train_eda_fin, window = 10, min_count=7, workers=6, size = 400, iter=30, sg=1, callbacks=[epoch_logger])
#min_count가 클수록, size가 작을수록 시간 단축인듯(정확도는...)
#min_count = 7, size = 400, window = 10일 때 걸린 시간: 대략 반복당 2분
#window size는 10개정도가 적당하다고 함(어느정도까지는 많을수록 좋음)
#size는 10~100 정도가 적당하다고 함(차원이 클수록 오래걸리고, 차원이 높다고 항상 좋지는 않다고 함, 수십~수백 굿)
#workers는 컴터 스펙에 따라 다른데 보통 4-6
#min_count는 주로 10-100 사이인듯

In [ ]:
wv_model.wv.most_similar(positive = ['아침', '눈', '크리스마스'],topn = 100)

In [ ]:
valid=pd.read_json('val.json')
valid

NameError: ignored

In [ ]:
def re_sub(series: pd.Series) -> pd.Series:
  series = series.str.replace(pat=r'[ㄱ-ㅎ]', repl=r'', regex=True) #ㅋ 제거용
  series = series.str.replace(pat=r'[^\w\s]', repl=r'', regex=True) #특수문자 제거용
  series = series.str.replace(pat=r'[ ]{2,}', repl=r' ', regex=True) #공백 제거
  series = series.str.replace(pat=r'[\u3000]+', repl=r'', regex=True) #u3000제거
  return series

def flatten(list_of_list : List) -> List:
  flatten = [j for i in list_of_list for j in i]
  return flatten

def get_token(title: str, tokenizer)-> List[Tuple]:

  if len(title)==0 or title== ' ' : #제목이 공백인 경우 tokenizer 에러 발생
    return []
  
  result = tokenizer.analyze(title)
  result = [(morph.lex, morph.tag) for split in result for morph in split.morphs] # (형태소, 품사) 튜플의 리스트
  return result

def get_all_tags(df) -> List:
  tag_list = df['tags'].values.tolist()
  tag_list=flatten(tag_list)
  return tag_list

tokenizer = KhaiiiApi()
all_tag = get_all_tags(valid)
token_tag = [get_token(x, tokenizer) for x in all_tag] #태그를 형태소 분석

token_tag[:10]


In [ ]:
token_itself=list(filter(lambda x: len(x)==1, token_tag)) # 태그 자체가 형태소여서 분리되지 않는 태그만 고름
token_itself=flatten(token_itself)
flatten_token=flatten(token_tag)
print('%-23s'%'# of original tag is', f'{len(all_tag):8,}')
print('%-23s'%'# of morpheme itself is', f'{len(token_itself):8,}')
print('%-23s'%'# of total token is', f'{len(flatten_token):8,}')


In [ ]:
#counting part of speech (pos)
pos=[x[1] for x in token_itself]
pos_count=Counter(pos)
popular_pos = pos_count.most_common()
#tag 분류표
objects=[x[0] for x in popular_pos]
y_pos=np.arange(len(objects))
performance=[x[1] for x in popular_pos]
plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Usage')
plt.title('Part of Speech - Tags')
plt.show()


In [ ]:
#플레이리스트 제목 형태소 분석
valid['plylst_title']=re_sub(valid['plylst_title'])
valid.loc[:, 'ply_token']=valid['plylst_title'].map(lambda x: get_token(x, tokenizer))
using_pos=['NNG', 'SL', 'NNP', 'MAG', 'SN'] #일반 명사, 외국어, 고유 명사, 일반 부사, 숫자
valid['ply_token']=valid['ply_token'].map(lambda x: list(filter(lambda x: x[1] in using_pos, x)))
unique_tag=set(token_itself)
unique_word=[x[0] for x in unique_tag]
#train 데이터의 plylst title 형태소 분리
#정답 tag에 나온 형태소만 남겨두기
valid['ply_token']=valid['ply_token'].map(lambda x: list(filter(lambda x: x[0] in unique_word, x)))
valid.head(10)

In [ ]:
plylst_date = valid[['updt_date', 'songs']]
plylst_date_unnest = np.dstack(
    (
        np.repeat(plylst_date.updt_date.values, list(map(len, plylst_date.songs))), 
        np.concatenate(plylst_date.songs.values)
    )
)
plylst_date = pd.DataFrame(data=plylst_date_unnest[0], columns = plylst_date.columns)

del plylst_date_unnest
plylst_date["updt_date"] = plylst_date["updt_date"].map(lambda x: x[:4] + x[5:7] + x[8:10])
song_date = song_meta[["id","issue_date"]]
plylst_song_date = pd.merge(plylst_date, song_date, left_on="songs", right_on="id", how='left')
plylst_song_date["issue_date"] = plylst_song_date["issue_date"].astype(str)

plylst_song_date["strange"] = plylst_song_date["updt_date"] < plylst_song_date["issue_date"]
strange_songs = plylst_song_date[plylst_song_date["strange"] == True].drop('songs', axis=1)
strange_songs=strange_songs.drop_duplicates(['id'])
strange_songs_list=strange_songs['id'].values.astype(str)
strange_song_gnr_map=song_gnr_map[song_gnr_map['song_id'].isin(strange_songs_list)] 
strange_cnt=strange_song_gnr_map.groupby('gnr_code').count()
strange_cnt=pd.merge(strange_cnt, gnr_code, on='gnr_code')
strange_id=strange_songs.loc[:, 'id']
strange_id
strange_id_list=list(strange_id)
valid_id=valid.loc[:, 'id']
song_meta_id=song_meta.loc[:, 'id']
#플레이리스트 아이디(id)와 수록곡(songs) 추출
plylst_song_map=valid[['id', 'songs']]

#unnest songs

plylst_song_map_unnest=np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))),
        np.concatenate(plylst_song_map.songs.values)
    )
)

#unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map=pd.DataFrame(data=plylst_song_map_unnest[0], columns=plylst_song_map.columns)
plylst_song_map['id']=plylst_song_map['id'].astype(str)
plylst_song_map['songs']=plylst_song_map['songs'].astype(int).astype(str)

del plylst_song_map_unnest
#플레이리스트에 strange 곡 제거한 후, 수록된 곡의 총 개수(중복 포함)

plylst_song_map_remove=plylst_song_map[np.logical_not(plylst_song_map.songs.isin(strange_songs_list))] 
plylst_song_map_remove.shape
#valid set에 새로운 column : remove_songs 생성
#remove_songs : strange 곡 제거한 후 다시 만든 플레이리스트

plylst_song_group=plylst_song_map_remove.groupby('id')['songs'].apply(list).reset_index(name='remove_songs')
valid['id']=valid['id'].astype(str)
valid_remove = pd.merge(valid, plylst_song_group)


In [ ]:
valid_remove

In [ ]:
y = []
for i in range(len(valid['songs'])):
  x = len(valid['songs'][i])==0
  y.insert(i,x)
A = valid[y]
A

In [ ]:
A['remove_songs'] = A['songs']
A

In [ ]:
valid_eda = pd.concat([valid_remove, A])
valid_eda

In [ ]:
val_sent = valid_eda[['id', 'ply_token', 'tags', 'remove_songs']]
val_sent

In [ ]:
val_sent = val_sent.reset_index(drop=True)

In [ ]:
val_sent = pd.DataFrame(val_sent)
val_sent

In [ ]:
#Word2vec 이용을 위해 ply_token data를 전처리
z = []
y = []
for i in range(len(val_sent['ply_token'])):
  z = []
  if len(val_sent['ply_token'][i]) == 0:
    x = val_sent['ply_token'][i]
    z = x
  else:
    for j in range(len(val_sent['ply_token'][i])):
     x = val_sent['ply_token'][i][j]
     z.insert(j,x[0])
  y.insert(i,z)
y

In [ ]:
val_sent['ply_token_new'] = y
val_sent = val_sent[['ply_token_new', 'tags', 'remove_songs']]
val_sent # Word2vec 이용을 위해 필요한 data들만 모아둔 dataframe


In [ ]:
#Word2vec 사용을 위해 플레이리스트 별로 제목의 형태소와 태그, 단어를 한 문장으로 갖도록 전처리
w = []
for i in range(len(val_sent)):
  w.insert(i,val_sent['ply_token_new'][i] + val_sent['tags'][i] + val_sent['remove_songs'][i])
val_input = w
val_input


In [ ]:
for i in range(len(val_input)

In [ ]:
wv_model.wv.most_similar(positive = val_input[0],topn = 100)

In [ ]:
# input: 플레이리스트 제목 or 곡 or 플레이리스트 제목&태그 or 곡&태그
wv_model.wv.most_similar('',topn = #뽑힐 개수) #input이 여러 개면 ''대신 positive=['', '', ...]

# 필요한 output: 곡 100개 & 태그 10개
# x는 ['', '', ...]의 형식
def result(x):
  if wv_model.wv.most_similar(positive=x, topn = 100)[]


while 
if 